# Vision support
https://platform.claude.com/docs/en/build-with-claude/vision

In [17]:
import base64
from os import environ
from typing import List, Dict, Iterable, Union, Any

# Load env variables and create client
from anthropic import AnthropicVertex
from anthropic.types import TextBlockParam, ImageBlockParam, Message, ToolParam, ContentBlock, \
    Base64ImageSourceParam

region = environ.get("CLOUD_ML_REGION", "")
project_id = environ.get("ANTHROPIC_VERTEX_PROJECT_ID", "")
client = AnthropicVertex(region=region, project_id=project_id)
model = "claude-sonnet-4-5@20250929"

In [18]:
# Helper functions
def add_user_message(messages: List[dict], msg: Any):
    """Adds a user message to the messages list."""
    user_message = {
        "role": "user",
        "content": msg.content if isinstance(msg, Message) else msg,
    }
    messages.append(user_message)


def add_assistant_message(messages: List[dict], msg: Union[Message, dict, str]):
    """Adds an assistant message to the messages list."""
    assistant_message = {
        "role": "assistant",
        "content": msg.content if isinstance(msg, Message) else msg,
    }
    messages.append(assistant_message)


def chat(
        messages: List[dict],
        system: str = None,
        temperature: float = 1.0,
        stop_sequences: Iterable[str] = [],
        tools: Iterable[ToolParam] = None,
        tool_choice: Dict = None,
        thinking: bool = False,
        thinking_budget: int = 1024,
):
    """Sends a chat request to the Claude model with the given messages."""
    params = {
        "model": model,
        "max_tokens": 4000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system is not None:
        params["system"] = system

    if tools is not None:
        params["tools"] = tools

    if tool_choice is not None:
        params["tool_choice"] = tool_choice

    if thinking:
        params["thinking"] = {
            "type": "enabled",
            "budget_tokens": thinking_budget,
        }

    message = client.messages.create(**params)
    return message


def text_from_message(message: Message) -> Iterable[str]:
    """Extracts and concatenates all text blocks from a Message object."""
    return "\n".join([block.text for block in message.content if block.type == "text"])

In [19]:
prompt = """
Analyze the attached satellite image of a property with these specific steps:

1. Residence identification: Locate the primary residence on the property by looking for:
   - The largest roofed structure
   - Typical residential features (driveway connection, regular geometry)
   - Distinction from other structures (garages, sheds, pools)

2. Tree overhang analysis: Examine all trees near the primary residence:
   - Identify any trees whose canopy extends directly over any portion of the roof
   - Estimate the percentage of roof covered by overhanging branches (0-25%, 25-50%, 50-75%, 75%+)
   - Note particularly dense areas of overhang

3. Fire risk assessment: For any overhanging trees, evaluate:
   - Potential wildfire vulnerability (ember catch points, continuous fuel paths to structure)
   - Proximity to chimneys, vents, or other roof openings if visible
   - Areas where branches create a "bridge" between wildland vegetation and the structure

4. Defensible space identification: Assess the property's overall vegetative structure:
   - Identify if trees connect to form a continuous canopy over or near the home
   - Note any obvious fuel ladders (vegetation that can carry fire from ground to tree to roof)

5. Fire risk rating: Based on your analysis, assign a Fire Risk Rating from 1-4:
   - Rating 1 (Low Risk): No tree branches overhanging the roof, good defensible space around
   - Rating 2 (Moderate Risk): Minimal overhang (<25% of roof), some separation between tree canopies
   - Rating 3 (High Risk): Significant overhang (25-50% of roof), connected tree canopies, multiple points of vulnerability
   - Rating 4 (Severe Risk): Extensive overhang (>50% of roof), dense vegetation against structure

For each item above (1-5), write one sentence summarizing your findings, with your final response being the numerical rating.
 """

In [27]:
# Optional: Add strict runtime validation using Pydantic TypeAdapter
from pydantic import TypeAdapter, ValidationError

# Create a validator for the image source
ImageSourceValidator = TypeAdapter(Base64ImageSourceParam)
ImageBlockValidator = TypeAdapter(ImageBlockParam)
TextBlockValidator = TypeAdapter(TextBlockParam)
ContentBlockValidator = TypeAdapter(ContentBlock)


def validate_content_blocks(blocks: List[dict]) -> List[ContentBlock]:
    """Validates a list of content blocks against the ContentBlock schema."""
    validated_blocks = []
    for block in blocks:
        try:
            validated_block = ContentBlockValidator.validate_python(block, by_name=True)
            validated_blocks.append(validated_block)
        except ValidationError as e:
            print(f"Content block validation error: {e}")
            raise
    return validated_blocks


def validate_image_source(source: dict) -> Base64ImageSourceParam:
    """Validates an image source dict against the Base64ImageSourceParam schema."""
    try:
        return ImageSourceValidator.validate_python(source, by_name=True)
    except ValidationError as e:
        print(f"Image source validation error: {e}")
        raise


def validate_image_block(block: dict) -> ImageBlockParam:
    """Validates an image block dict against the ImageBlockParam schema."""
    try:
        return ImageBlockValidator.validate_python(block, by_name=True)
    except ValidationError as e:
        print(f"Image block validation error: {e}")
        raise


def validate_text_block(block: dict) -> TextBlockParam:
    """Validates a text block dict against the TextBlockParam schema."""
    try:
        return TextBlockValidator.validate_python(block, by_name=True)
    except ValidationError as e:
        print(f"Text block validation error: {e}")
        raise


In [29]:
with open("006-images/prop7.png", "rb") as f:
    image_data = base64.standard_b64encode(f.read()).decode('utf-8')

# Create properly typed and validated blocks
image_source: Base64ImageSourceParam = {
    "type": "base64",
    "media_type": "image/png",
    "data": image_data,
}

image_block: ImageBlockParam = {
    "type": "image",
    "source": image_source,
}

text_block: TextBlockParam = {
    "type": "text",
    "text": prompt,
}

# Create the messages list with validated content
blocks: List[dict] = [validate_image_block(image_block), validate_text_block(text_block)]
# Serialize Pydantic models back to dicts for API compatibility
serialized_blocks = [block.model_dump(mode='python') if hasattr(block, 'model_dump') else block for block in
                     blocks]
add_user_message(messages, serialized_blocks)

[{'role': 'user',
  'content': [{'type': 'image',
    'source': {'type': 'base64',
     'media_type': 'image/png',
     'data': '...'}},
   {'type': 'text',
    'text': '\nAnalyze the attached satellite image of a property with these specific steps:\n\n1. Residence identification: Locate the primary residence on the property by looking for:\n   - The largest roofed structure\n   - Typical residential features (driveway connection, regular geometry)\n   - Distinction from other structures (garages, sheds, pools)\n\n2. Tree overhang analysis: Examine all trees near the primary residence:\n   - Identify any trees whose canopy extends directly over any portion of the roof\n   - Estimate the percentage of roof covered by overhanging branches (0-25%, 25-50%, 50-75%, 75%+)\n   - Note particularly dense areas of overhang\n\n3. Fire risk assessment: For any overhanging trees, evaluate:\n   - Potential wildfire vulnerability (ember catch points, continuous fuel paths to structure)\n   - Proximit

In [30]:
rest = chat(messages)
text_from_message(rest)

'# Fire Risk Analysis\n\n1. **Residence Identification**: The primary residence is the large, light-colored structure with a complex roof geometry located in the center of the image, clearly distinguishable as the main dwelling with multiple roof planes and connected sections.\n\n2. **Tree Overhang Analysis**: Multiple large tree canopies directly overhang significant portions of the residence roof, with particularly dense coverage on the eastern and northern sections, estimating approximately 40-50% of the total roof area is covered by overhanging branches.\n\n3. **Fire Risk Assessment**: The overhanging trees create severe wildfire vulnerability with multiple ember catch points across the roof surface, direct contact points between vegetation and structure, and continuous fuel pathways from the surrounding dense forest canopy directly to the home.\n\n4. **Defensible Space Identification**: The property shows poor defensible space with a nearly continuous tree canopy surrounding and c